# Desafio_FIESC: Agente Industrial Inteligente

Neste notebook iremos:
1. Conectar ao SQLite
2. Definir parser NL → SQL
3. Testar consultas em linguagem natural


### A seguir, um diagrama de alto nível da solução, demonstrando como os componentes interagem:

flowchart TD
  %% Usuário interage com a interface
  U[Usuário] -->|Faz pergunta NL| UI[Front-end (Streamlit/Notebook)]

  %% UI encaminha para o parser
  UI -->|Pergunta| Parser[Camada de Parsing NL→SQL]

  %% Parser usa contexto e gera SQL
  Parser -->|SQL gerado| Executor[Camada de Execução SQL]
  subgraph Contexto
    Ctx[Memória de Contexto]
    Parser -->|Consulta/dependência| Ctx
    Ctx -->|Dados de contexto| Parser
  end

  %% Execução contra o banco
  Executor -->|Query SQL| DB[(SQLite Database)]
  DB -->|Resultados| Executor

  %% Resultados retornam e são exibidos
  Executor -->|DataFrame/JSON| UI
  UI -->|Mostra resposta| U

In [5]:
%pip install pandas
# Importa pacotes necessários
import sqlite3           # para conexão com SQLite
import pandas as pd      # para manipulação de tabelos retornados

# Conecta ao banco (arquivo .db deve estar na mesma pasta do notebook)
conn = sqlite3.connect('manutencao_industrial.db')
print("✅ Conectado ao SQLite:", conn)

Note: you may need to restart the kernel to use updated packages.
✅ Conectado ao SQLite: <sqlite3.Connection object at 0x000001A289949D40>


## Inspeção da Estrutura do Banco de Dados

A célula a seguir apresenta uma função que lista todas as tabelas do banco e exibe, para cada uma, suas colunas, tipos de dados e indicações de chave primária.  
Isso ajuda a entender rapidamente o schema disponível antes de criarmos novas regras de mapeamento NL → SQL.



In [6]:
def mostrar_schema(conn):
    """
    Exibe o schema completo do banco SQLite conectado em `conn`.
    
    Passos internos:
    1. Cria um cursor para executar comandos SQL.
    2. Consulta sqlite_master para listar todas as tabelas.
    3. Para cada tabela:
       a) Imprime o nome da tabela.
       b) Executa PRAGMA table_info para obter metadados de colunas.
       c) Converte o resultado em um DataFrame pandas para fácil visualização.
       d) Ajusta os nomes das colunas do DataFrame para torná-las legíveis.
       e) Exibe o DataFrame e adiciona uma linha em branco para separar saídas.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tabelas = [t[0] for t in cursor.fetchall()]
    for table in tabelas:
        print(f"🗄️ Tabela: {table}")
        # PRAGMA table_info retorna a estrutura de colunas
        info = pd.read_sql(f"PRAGMA table_info({table});", conn)
        info.columns = ["cid", "nome_coluna", "tipo", "notnull", "dflt_value", "pk"]
        display(info)  # mostra o DataFrame com o schema
        print()       # linha em branco
mostrar_schema(conn)


🗄️ Tabela: equipamentos


,cid,nome_coluna,tipo,notnull,dflt_value,pk
0,0,id_equipamento,INTEGER,0,None,1
1,1,tipo,TEXT,0,None,0
2,2,localizacao,TEXT,0,None,0
3,3,status,TEXT,0,None,0



🗄️ Tabela: ordens_manutencao


,cid,nome_coluna,tipo,notnull,dflt_value,pk
0,0,id_ordem,INTEGER,0,None,1
1,1,id_equipamento,INTEGER,0,None,0
2,2,data_abertura,DATE,0,None,0
3,3,data_conclusao,DATE,0,None,0
4,4,tipo_manutencao,TEXT,0,None,0
5,5,status,TEXT,0,None,0



🗄️ Tabela: tecnicos


,cid,nome_coluna,tipo,notnull,dflt_value,pk
0,0,id_tecnico,INTEGER,0,None,1
1,1,nome,TEXT,0,None,0
2,2,especialidade,TEXT,0,None,0
3,3,turno,TEXT,0,None,0



🗄️ Tabela: ordem_tecnico


,cid,nome_coluna,tipo,notnull,dflt_value,pk
0,0,id_ordem,INTEGER,0,None,1
1,1,id_tecnico,INTEGER,0,None,2


---
## Extensão do Parser: Novas Regras de Consulta

Na próxima célula vamos ampliar a função `nl_to_sql` para suportar consultas como:

1. **Ordens concluídas** — listar todas as ordens cujo status é “concluída”.  
2. **Ordens nos últimos N dias** — filtrar ordens pela data de abertura nos últimos dias.  
3. **Contagem de técnicos por especialidade** — agrupar técnicos por sua especialidade e contar quantos há em cada.  
4. **Histórico de uma ordem específica** — recuperar quais técnicos trabalharam em uma determinada ordem.

Cada bloco de código dentro da função virá comentado, explicando:
- O padrão de linguagem natural que acionou a regra.
- A construção da query SQL correspondente.
---


In [8]:
import re
from datetime import datetime, timedelta

def nl_to_sql(query: str) -> str:
    """
    Converte uma pergunta em linguagem natural para uma query SQL.
    Adiciona múltiplas regras para diferentes intenções.
    """
    q = query.lower()

    # 1) Equipamentos em manutenção
    if 'equipamentos' in q and 'manutenção' in q:
        return "SELECT * FROM equipamentos WHERE status LIKE '%manutenção%'"

    # 2) Ordens abertas em um setor específico
    if 'ordens abertas' in q:
        m = re.search(r'setor (\w+)', q)
        if m:
            setor = m.group(1).capitalize()
            return (
                "SELECT o.* "
                "FROM ordens_manutencao o "
                "JOIN equipamentos e ON o.id_equipamento = e.id_equipamento "
                f"WHERE o.status = 'aberta' AND e.localizacao = '{setor}'"
            )

    # 3) Técnicos de elétrica no turno noturno
    if 'técnicos' in q and 'elétrica' in q and 'noturno' in q:
        return "SELECT * FROM tecnicos WHERE especialidade = 'elétrica' AND turno = 'noturno'"

    # 4) Tempo médio de manutenção corretiva das bombas
    if 'tempo médio' in q and 'manutenção corretiva' in q and 'bombas' in q:
        return (
            "SELECT AVG(julianday(data_conclusao) - julianday(data_abertura)) AS media_dias "
            "FROM ordens_manutencao o "
            "JOIN equipamentos e ON o.id_equipamento = e.id_equipamento "
            "WHERE tipo_manutencao = 'corretiva' AND e.tipo = 'Bomba'"
        )

    # 5) Ordens concluídas
    # Ex.: “Liste as ordens concluídas”
    if 'ordens' in q and 'concluída' in q:
        return "SELECT * FROM ordens_manutencao WHERE status = 'concluída'"

    # 6) Ordens de manutenção nos últimos N dias
    # Ex.: “Quais ordens foram abertas nos últimos 7 dias?”
    m = re.search(r'últimos (\d+) dias', q)
    if 'ordens' in q and m:
        dias = int(m.group(1))
        data_limite = (datetime.now() - timedelta(days=dias)).strftime('%Y-%m-%d')
        return (
            "SELECT * FROM ordens_manutencao "
            f"WHERE date(data_abertura) >= '{data_limite}'"
        )

    # 7) Contagem de técnicos por especialidade
    # Ex.: “Quantos técnicos existem por especialidade?”
    if 'quantos técnicos' in q and 'especialidade' in q:
        return (
            "SELECT especialidade, COUNT(*) AS total_tecnicos "
            "FROM tecnicos "
            "GROUP BY especialidade"
        )

    # 8) Histórico de técnicos de uma ordem
    # Ex.: “Quem trabalhou na ordem 3?”
    m = re.search(r'ordem (\d+)', q)
    if 'quem' in q and m:
        ordem_id = m.group(1)
        return (
            "SELECT t.* "
            "FROM tecnicos t "
            "JOIN ordem_tecnico ot ON t.id_tecnico = ot.id_tecnico "
            f"WHERE ot.id_ordem = {ordem_id}"
        )

    # Se nenhuma regra foi acionada:
    return ""


In [10]:
# Teste da função (retorna a string SQL)
print(nl_to_sql("Quais equipamentos estão em manutenção?"))

SELECT * FROM equipamentos WHERE status LIKE '%manutenção%'


In [12]:
from IPython.display import display

def executar_query(nl: str):
    """
    Recebe uma pergunta em NL, gera SQL com nl_to_sql(),
    executa e exibe o DataFrame resultante.
    """
    sql = nl_to_sql(nl)
    if not sql:
        print(f"❌ Não consegui mapear a pergunta: «{nl}»")
        return
    print("🔍 SQL gerado:\n", sql)
    df = pd.read_sql(sql, conn)
    display(df)

In [20]:
# 1) Ordens concluídas
#executar_query("Liste as ordens concluídas")

# 2) Ordens abertas nos últimos 7 dias
#executar_query("Quais ordens foram abertas nos últimos 7 dias?")

# 3) Quantos técnicos existem por especialidade?
#executar_query("Quantos técnicos existem por especialidade?")
# 4) Quem trabalhou na ordem 3?
#executar_query("Quem trabalhou na ordem 3?")


🔍 SQL gerado:
 SELECT especialidade, COUNT(*) AS total_tecnicos FROM tecnicos GROUP BY especialidade


,especialidade,total_tecnicos
0,elétrica,2
1,hidráulica,2
2,mecânica,2
